In [1]:
import pandas as pd

/tmp/ipykernel_345700/4080736814.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [12]:
import pandas as pd
import re
for group in range(1,11):
    # Load the files
    oracle_path = f'Group_{group:02}_cga_Oracle.csv'
    dped_path = f'Group_{group:02}_DPed.csv'

    oracle_df = pd.read_csv(oracle_path)
    dped_df = pd.read_csv(dped_path)

    # Merge the two dataframes based on the 'Utterance' column
    # Keeping all columns from oracle_df and only the 'DPed' column from dped_df
    merged_df = pd.merge(oracle_df, dped_df[['Utterance', 'DPed']], on='Utterance', how='left')

    # Only update the 'Transcript' where 'DPed' is not NaN
    merged_df.loc[~merged_df['DPed'].isna(), 'Transcript'] = merged_df['DPed']

    # Drop the now redundant 'DPed' column
    merged_df.drop(columns=['DPed'], inplace=True)

    # Save the updated dataframe to a new CSV file
    updated_file_path = f'OracleFiles/Group_{group:02d}Oracle.csv'
    #merged_df.to_csv(updated_file_path, index=False)
    merged_df['Common Ground'] = merged_df['Common Ground'].astype(str)

    # Filtering rows where 'Common Ground' contains 'STATEMENT'
    statement_rows = merged_df[merged_df['Common Ground'].str.contains('STATEMENT')]
    print(statement_rows)

    # Define a function to extract content within brackets
    def extract_brackets(text):
        match = re.search(r'\((.*?)\)', text)
        if match:
            return match.group(1)
        else:
            return "No Match Found"  # Return a placeholder if no match is found


    # Apply the function to the 'Common Ground' column
    statement_rows.loc[:, 'Common Ground'] = statement_rows['Common Ground'].apply(extract_brackets)
    print(statement_rows)
    statement_rows.to_csv(updated_file_path)


     Utterance   Start     End  \
5            5   24.42   27.72   
24          24   78.08   80.58   
30          30   95.19   97.17   
31          31   97.56  102.03   
37          36  110.16  114.24   
40          36  110.16  114.24   
45          41  127.35  132.00   
47          41  127.35  132.00   
55          47  140.73  145.89   
68          57  174.54  185.08   
79          65  207.06  210.87   
92          75  234.27  239.82   
104         85  268.08  272.58   

                                         Common Ground  \
5                              S0: STATEMENT(red = 10)   
24                             S1: STATEMENT(red = 10)   
30                           S2: STATEMENT(red = blue)   
31                            S3: STATEMENT(blue = 10)   
37                       S5: STATEMENT: (purple != 20)   
40                          S4: STATEMENT(purple > 20)   
45   S7: STATEMENT(S209: STATEMENT(red = 10 and blu...   
47                           S6: STATEMENT(green = 20)   
5